In [ ]:
%%capture
import os
import site
os.sys.path.insert(0, '/home/schirrmr/code/reversible/reversible2/')
os.sys.path.insert(0, '/home/schirrmr/braindecode/code/braindecode/')
os.sys.path.insert(0, '/home/schirrmr/code/explaining/reversible//')
%cd /home/schirrmr/


%load_ext autoreload
%autoreload 2
import numpy as np
import logging
log = logging.getLogger()
log.setLevel('INFO')
import sys
logging.basicConfig(format='%(asctime)s %(levelname)s : %(message)s',
                     level=logging.INFO, stream=sys.stdout)
import matplotlib
from matplotlib import pyplot as plt
from matplotlib import cm
%matplotlib inline
%config InlineBackend.figure_format = 'png'
matplotlib.rcParams['figure.figsize'] = (12.0, 1.0)
matplotlib.rcParams['font.size'] = 14
import seaborn
seaborn.set_style('darkgrid')

from reversible.sliced import sliced_from_samples

from numpy.random import RandomState

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
import numpy as np
import copy
import math

import itertools
from reversible.plot import create_bw_image
import torch as th
from braindecode.torch_ext.util import np_to_var, var_to_np
from reversible.revnet import ResidualBlock, invert, SubsampleSplitter, ViewAs, ReversibleBlockOld
from spectral_norm import spectral_norm
from conv_spectral_norm import conv_spectral_norm

def display_text(text, fontsize=18):
    fig = plt.figure(figsize=(12,0.1))
    plt.title(text, fontsize=fontsize)
    plt.axis('off')
    display(fig)
    plt.close(fig)

In [ ]:
rng = RandomState(39482014)
plt.plot(np.sin(np.linspace(0, 2*np.pi, endpoint=False, num=8)))
a = np.sin(np.linspace(0, 2*np.pi, endpoint=False, num=8))
amps_a = rng.randn(500) + 6
phases_a = rng.rand(500) * 2 * np.pi 
signals = np.array([amp * np.sin(np.linspace(0, 2*np.pi, endpoint=True, num=8) + phase) 
     for amp, phase in zip(amps_a, phases_a)])
plt.figure()
plt.plot(signals[:10].squeeze().T)
inputs = [np_to_var(signals[:,None,:,None], dtype=np.float32)]

In [ ]:
ffted = np.fft.rfft(signals)

i_dim = 1
x1 = np.real(ffted)[:,i_dim]
x2 = np.imag(ffted)[:,i_dim]

plt.figure(figsize=(4,4))
plt.plot(x1,x2, ls='', marker='o')
inputs = np_to_var(np.stack((x1,x2), axis=1), dtype=np.float32)
plt.figure(figsize=(4,4))
plt.plot(var_to_np(inputs)[:,0],var_to_np(inputs)[:,1], ls='', marker='o')

### gaussian and gaussian

In [ ]:

def get_samples(n_samples):
    mean = means_per_cluster[0]
    std = th.exp(stds_per_cluster[0])

    amps = get_gauss_samples(n_samples, mean[:1], std[:1], truncate_to=3)
    phases = get_gauss_samples(n_samples, mean[1:], std[1:], truncate_to=3)

    x, y = amp_phase_to_x_y(amps, phases)
    return th.cat((x,y), dim=1)

n_dims=2
n_clusters = 1
means_per_cluster = [th.autograd.Variable(th.zeros(n_dims), requires_grad=True)
                     for _ in range(n_clusters)]
# keep in mind this is in log domain so 0 is std 1
stds_per_cluster = [th.autograd.Variable(th.zeros(n_dims), requires_grad=True)
                    for _ in range(n_clusters)]


optim_dist = th.optim.Adam(
                          [
    {'params': means_per_cluster + stds_per_cluster,
    'lr': 1e-2,
    'weight_decay': 0},], betas=(0,0.9))

In [ ]:
from reversible.gaussian import get_gauss_samples
from ampphase import amp_phase_to_x_y
from reversible.sliced import sliced_from_samples
n_samples = 500
n_epochs = 10000
for i_epoch in range(n_epochs):
    samples = get_samples(n_samples)
    sliced_loss = sliced_from_samples(inputs, samples, 10, None)
    optim_dist.zero_grad()
    sliced_loss.backward()
    optim_dist.step()
    if i_epoch % (n_epochs // 20) == 0:
        print("sliced_loss: {:.2f}".format(sliced_loss.item()))
        print(means_per_cluster)
        print(th.exp(stds_per_cluster[0]))
        fig = plt.figure(figsize=(5,5))
        plt.plot(var_to_np(samples)[:,0],var_to_np(samples)[:,1], ls='', marker='o')
        display(fig)
        plt.close(fig)


In [ ]:
from reversible.uniform import get_uniform_samples
def get_samples(n_samples):
    mean = means_per_cluster[0]
    std = th.exp(stds_per_cluster[0])

    amps = get_gauss_samples(n_samples, mean[:1], std[:1], truncate_to=3)
    phases = get_uniform_samples(n_samples, mean[1:], std[1:] * 2 * np.pi)

    x, y = amp_phase_to_x_y(amps, phases)
    return th.cat((x,y), dim=1)

n_dims=2
n_clusters = 1
means_per_cluster = [th.autograd.Variable(th.zeros(n_dims), requires_grad=True)
                     for _ in range(n_clusters)]
# keep in mind this is in log domain so 0 is std 1
stds_per_cluster = [th.autograd.Variable(th.zeros(n_dims), requires_grad=True)
                    for _ in range(n_clusters)]


optim_dist = th.optim.Adam(
                          [
    {'params': means_per_cluster + stds_per_cluster,
    'lr': 1e-2,
    'weight_decay': 0},], betas=(0,0.9))

In [ ]:
from reversible.gaussian import get_gauss_samples
from ampphase import amp_phase_to_x_y
from reversible.sliced import sliced_from_samples
n_samples = 500
n_epochs = 10000
for i_epoch in range(n_epochs):
    samples = get_samples(n_samples)
    sliced_loss = sliced_from_samples(inputs, samples, 10, None)
    optim_dist.zero_grad()
    sliced_loss.backward()
    optim_dist.step()
    if i_epoch % (n_epochs // 20) == 0:
        print("sliced_loss: {:.2f}".format(sliced_loss.item()))
        print(means_per_cluster)
        print(th.exp(stds_per_cluster[0]))
        fig = plt.figure(figsize=(5,5))
        plt.plot(var_to_np(samples)[:,0],var_to_np(samples)[:,1], ls='', marker='o')
        display(fig)
        plt.close(fig)


In [ ]:
from reversible.uniform import get_uniform_samples
def get_samples(n_samples):
    mean = means_per_cluster[0]
    std = th.exp(stds_per_cluster[0])

    amps = get_gauss_samples(n_samples, mean[:1], std[:1], truncate_to=3)
    phases = get_uniform_samples(n_samples, mean[1:], std[1:] * 2 * np.pi)

    x, y = amp_phase_to_x_y(amps, phases)
    return th.cat((x,y), dim=1)

n_dims=2
n_clusters = 1
means_per_cluster = [th.autograd.Variable(th.zeros(n_dims), requires_grad=True)
                     for _ in range(n_clusters)]
# keep in mind this is in log domain so 0 is std 1
stds_per_cluster = [th.autograd.Variable(th.zeros(n_dims), requires_grad=True)
                    for _ in range(n_clusters)]

means_per_cluster[0].data[0] = 23
stds_per_cluster[0].data[0] = 1

optim_dist = th.optim.Adam(
                          [
    {'params': means_per_cluster + stds_per_cluster,
    'lr': 1e-2,
    'weight_decay': 0},], betas=(0,0.9))

samples = get_samples(n_samples)
plt.figure(figsize=(5,5))
plt.plot(var_to_np(samples)[:,0],var_to_np(samples)[:,1], ls='', marker='o')
plt.plot(var_to_np(inputs)[:,0],var_to_np(inputs)[:,1], ls='', marker='o')
sliced_loss = sliced_from_samples(inputs, samples, 10, None)
print(sliced_loss)

In [ ]:
from reversible.gaussian import get_gauss_samples
from ampphase import amp_phase_to_x_y
from reversible.sliced import sliced_from_samples
n_samples = 500
n_epochs = 1000
for i_epoch in range(n_epochs):
    samples = get_samples(n_samples)
    sliced_loss = sliced_from_samples(inputs, samples, 10, None)
    optim_dist.zero_grad()
    sliced_loss.backward()
    optim_dist.step()
    if i_epoch % (n_epochs // 20) == 0:
        print("sliced_loss: {:.2f}".format(sliced_loss.item()))
        print(means_per_cluster)
        print(th.exp(stds_per_cluster[0]))
        fig = plt.figure(figsize=(5,5))
        plt.plot(var_to_np(samples)[:,0],var_to_np(samples)[:,1], ls='', marker='o')
        plt.plot(var_to_np(inputs)[:,0],var_to_np(inputs)[:,1], ls='', marker='o')
        display(fig)
        plt.close(fig)



In [ ]:
# match in amplitude phase space:
# amplitudes simple match
# phase first compute mean phase, then use it to offset the phases, then match

### amp phase matching

In [ ]:
from reversible.uniform import get_uniform_samples
def get_samples(n_samples):
    mean = means_per_cluster[0]
    std = th.exp(stds_per_cluster[0])

    amps = get_gauss_samples(n_samples, mean[:1], std[:1], truncate_to=3)
    phases = get_uniform_samples(n_samples, mean[1:], std[1:] * 2 * np.pi)

    x, y = amp_phase_to_x_y(amps, phases)
    return th.cat((x,y), dim=1)

n_dims=2
n_clusters = 1
means_per_cluster = [th.autograd.Variable(th.zeros(n_dims), requires_grad=True)
                     for _ in range(n_clusters)]
# keep in mind this is in log domain so 0 is std 1
stds_per_cluster = [th.autograd.Variable(th.zeros(n_dims), requires_grad=True)
                    for _ in range(n_clusters)]

means_per_cluster[0].data[0] = 23
stds_per_cluster[0].data[0] = 1

optim_dist = th.optim.Adam(
                          [
    {'params': means_per_cluster + stds_per_cluster,
    'lr': 1e-2,
    'weight_decay': 0},], betas=(0,0.9))

samples = get_samples(n_samples)
plt.figure(figsize=(5,5))
plt.plot(var_to_np(samples)[:,0],var_to_np(samples)[:,1], ls='', marker='o')
plt.plot(var_to_np(inputs)[:,0],var_to_np(inputs)[:,1], ls='', marker='o')
sliced_loss = sliced_from_samples(inputs, samples, 10, None)
print(sliced_loss)

from ampphase import outs_to_amp_phase, to_amp_phase, amp_phase_to_x_y

def set_phase_interval_around_mean(phases, mean_phase):
    mean_phase = mean_phase.unsqueeze(0)
    return ((phases - mean_phase + np.pi) % (2 * np.pi)) - np.pi + mean_phase


i_amp, i_phase = to_amp_phase(inputs[:,:1],inputs[:,1:])
i_amp = i_amp.clone().detach()
i_phase = i_phase.clone().detach()

In [ ]:
from reversible.gaussian import get_gauss_samples
from ampphase import amp_phase_to_x_y
from reversible.sliced import sliced_from_samples
n_samples = 500
n_epochs = 1000
for i_epoch in range(n_epochs):
    samples = get_samples(n_samples)
    mean_sample = (th.mean(samples, dim=0, keepdim=True) + th.mean(inputs, dim=0, keepdim=True)) / 2
    _, mean_phase = to_amp_phase(mean_sample[:,:1], mean_sample[:,1:])

    s_amp, s_phase = to_amp_phase(samples[:,:1], samples[:,1:])
    s_phase = set_phase_interval_around_mean(s_phase, mean_phase.squeeze())
    i_amp, i_phase = to_amp_phase(inputs[:,:1],inputs[:,1:])

    i_phase = set_phase_interval_around_mean(i_phase, mean_phase.squeeze())

    sliced_amp = sliced_from_samples(s_amp, i_amp, 10, None)
    sliced_phase = sliced_from_samples(s_phase, i_phase, 10, None)

    
    sliced_loss = sliced_amp + sliced_phase
    optim_dist.zero_grad()
    sliced_loss.backward()
    optim_dist.step()
    if i_epoch % (n_epochs // 20) == 0:
        print("sliced_loss: {:.2f}".format(sliced_loss.item()))
        print("sliced_amp: {:.2f}".format(sliced_amp.item()))
        print("sliced_phase: {:.2f}".format(sliced_phase.item()))
        print(means_per_cluster)
        print(th.exp(stds_per_cluster[0]))
        fig = plt.figure(figsize=(3,3))
        plt.plot(var_to_np(samples)[:,0],var_to_np(samples)[:,1], ls='', marker='o')
        plt.plot(var_to_np(inputs)[:,0],var_to_np(inputs)[:,1], ls='', marker='o')
        display(fig)
        plt.close(fig)



### ampphase matching bad init

In [ ]:
from reversible.uniform import get_uniform_samples
def get_samples(n_samples):
    mean = means_per_cluster[0]
    std = th.exp(stds_per_cluster[0])

    amps = get_gauss_samples(n_samples, mean[:1], std[:1], truncate_to=3)
    phases = get_uniform_samples(n_samples, mean[1:], std[1:] * 2 * np.pi)

    x, y = amp_phase_to_x_y(amps, phases)
    return th.cat((x,y), dim=1)

n_dims=2
n_clusters = 1
means_per_cluster = [th.autograd.Variable(th.zeros(n_dims), requires_grad=True)
                     for _ in range(n_clusters)]
# keep in mind this is in log domain so 0 is std 1
stds_per_cluster = [th.autograd.Variable(th.zeros(n_dims), requires_grad=True)
                    for _ in range(n_clusters)]


optim_dist = th.optim.Adam(
                          [
    {'params': means_per_cluster + stds_per_cluster,
    'lr': 1e-2,
    'weight_decay': 0},], betas=(0,0.9))

samples = get_samples(n_samples)
plt.figure(figsize=(5,5))
plt.plot(var_to_np(samples)[:,0],var_to_np(samples)[:,1], ls='', marker='o')
plt.plot(var_to_np(inputs)[:,0],var_to_np(inputs)[:,1], ls='', marker='o')
sliced_loss = sliced_from_samples(inputs, samples, 10, None)
print(sliced_loss)

from ampphase import outs_to_amp_phase, to_amp_phase, amp_phase_to_x_y

def set_phase_interval_around_mean(phases, mean_phase):
    mean_phase = mean_phase.unsqueeze(0)
    return ((phases - mean_phase + np.pi) % (2 * np.pi)) - np.pi + mean_phase


i_amp, i_phase = to_amp_phase(inputs[:,:1],inputs[:,1:])
i_amp = i_amp.clone().detach()
i_phase = i_phase.clone().detach()

In [ ]:
from reversible.gaussian import get_gauss_samples
from ampphase import amp_phase_to_x_y
from reversible.sliced import sliced_from_samples
n_samples = 500
n_epochs = 10000
for i_epoch in range(n_epochs):
    samples = get_samples(n_samples)
    mean_sample = (th.mean(samples, dim=0, keepdim=True) + th.mean(inputs, dim=0, keepdim=True)) / 2
    _, mean_phase = to_amp_phase(mean_sample[:,:1], mean_sample[:,1:])

    s_amp, s_phase = to_amp_phase(samples[:,:1], samples[:,1:])
    s_phase = set_phase_interval_around_mean(s_phase, mean_phase.squeeze())
    i_amp, i_phase = to_amp_phase(inputs[:,:1],inputs[:,1:])

    i_phase = set_phase_interval_around_mean(i_phase, mean_phase.squeeze())

    sliced_amp = sliced_from_samples(s_amp, i_amp, 10, None)
    sliced_phase = sliced_from_samples(s_phase, i_phase, 10, None)

    
    sliced_loss = sliced_amp + sliced_phase
    optim_dist.zero_grad()
    sliced_loss.backward()
    optim_dist.step()
    if i_epoch % (n_epochs // 20) == 0:
        print("sliced_loss: {:.2f}".format(sliced_loss.item()))
        print("sliced_amp: {:.2f}".format(sliced_amp.item()))
        print("sliced_phase: {:.2f}".format(sliced_phase.item()))
        print(means_per_cluster)
        print(th.exp(stds_per_cluster[0]))
        fig = plt.figure(figsize=(3,3))
        plt.plot(var_to_np(samples)[:,0],var_to_np(samples)[:,1], ls='', marker='o')
        plt.plot(var_to_np(inputs)[:,0],var_to_np(inputs)[:,1], ls='', marker='o')
        display(fig)
        plt.close(fig)



## absolute gaussian

In [ ]:
from reversible.uniform import get_uniform_samples
def get_samples(n_samples):
    mean = means_per_cluster[0]
    std = th.exp(stds_per_cluster[0])

    amps = th.abs(get_gauss_samples(n_samples, mean[:1], std[:1], truncate_to=3))
    phases = get_uniform_samples(n_samples, mean[1:], std[1:] * 2 * np.pi)

    x, y = amp_phase_to_x_y(amps, phases)
    return th.cat((x,y), dim=1)

n_dims=2
n_clusters = 1
means_per_cluster = [th.autograd.Variable(th.zeros(n_dims), requires_grad=True)
                     for _ in range(n_clusters)]
# keep in mind this is in log domain so 0 is std 1
stds_per_cluster = [th.autograd.Variable(th.zeros(n_dims), requires_grad=True)
                    for _ in range(n_clusters)]


optim_dist = th.optim.Adam(
                          [
    {'params': means_per_cluster + stds_per_cluster,
    'lr': 1e-2,
    'weight_decay': 0},], betas=(0,0.9))

samples = get_samples(n_samples)
plt.figure(figsize=(5,5))
plt.plot(var_to_np(samples)[:,0],var_to_np(samples)[:,1], ls='', marker='o')
plt.plot(var_to_np(inputs)[:,0],var_to_np(inputs)[:,1], ls='', marker='o')
sliced_loss = sliced_from_samples(inputs, samples, 10, None)
print(sliced_loss)

from ampphase import outs_to_amp_phase, to_amp_phase, amp_phase_to_x_y

def set_phase_interval_around_mean(phases, mean_phase):
    mean_phase = mean_phase.unsqueeze(0)
    return ((phases - mean_phase + np.pi) % (2 * np.pi)) - np.pi + mean_phase


i_amp, i_phase = to_amp_phase(inputs[:,:1],inputs[:,1:])
i_amp = i_amp.clone().detach()
i_phase = i_phase.clone().detach()

In [ ]:
from reversible.gaussian import get_gauss_samples
from ampphase import amp_phase_to_x_y
from reversible.sliced import sliced_from_samples
n_samples = 500
n_epochs = 10000
for i_epoch in range(n_epochs):
    samples = get_samples(n_samples)
    mean_sample = (th.mean(samples, dim=0, keepdim=True) + th.mean(inputs, dim=0, keepdim=True)) / 2
    _, mean_phase = to_amp_phase(mean_sample[:,:1], mean_sample[:,1:])

    s_amp, s_phase = to_amp_phase(samples[:,:1], samples[:,1:])
    s_phase = set_phase_interval_around_mean(s_phase, mean_phase.squeeze())
    i_amp, i_phase = to_amp_phase(inputs[:,:1],inputs[:,1:])

    i_phase = set_phase_interval_around_mean(i_phase, mean_phase.squeeze())

    sliced_amp = sliced_from_samples(s_amp, i_amp, 10, None)
    sliced_phase = sliced_from_samples(s_phase, i_phase, 10, None)

    
    sliced_loss = sliced_amp + sliced_phase
    optim_dist.zero_grad()
    sliced_loss.backward()
    optim_dist.step()
    if i_epoch % (n_epochs // 20) == 0:
        print("sliced_loss: {:.2f}".format(sliced_loss.item()))
        print("sliced_amp: {:.2f}".format(sliced_amp.item()))
        print("sliced_phase: {:.2f}".format(sliced_phase.item()))
        print(means_per_cluster)
        print(th.exp(stds_per_cluster[0]))
        fig = plt.figure(figsize=(3,3))
        plt.plot(var_to_np(samples)[:,0],var_to_np(samples)[:,1], ls='', marker='o')
        plt.plot(var_to_np(inputs)[:,0],var_to_np(inputs)[:,1], ls='', marker='o')
        display(fig)
        plt.close(fig)



In [ ]:
from reversible.uniform import get_uniform_samples
def get_samples(n_samples):
    mean = means_per_cluster[0]
    std = th.exp(stds_per_cluster[0])

    amps = th.abs(get_gauss_samples(n_samples, mean[:1], std[:1], truncate_to=3)) + lower_bound_amp
    phases = get_uniform_samples(n_samples, mean[1:], std[1:] * 2 * np.pi)

    x, y = amp_phase_to_x_y(amps, phases)
    return th.cat((x,y), dim=1)

n_dims=2
n_clusters = 1
means_per_cluster = [th.autograd.Variable(th.zeros(n_dims), requires_grad=True)
                     for _ in range(n_clusters)]
# keep in mind this is in log domain so 0 is std 1
stds_per_cluster = [th.autograd.Variable(th.zeros(n_dims), requires_grad=True)
                    for _ in range(n_clusters)]
lower_bound_amp = th.zeros(1, requires_grad=True)


optim_dist = th.optim.Adam(
                          [
    {'params': means_per_cluster + stds_per_cluster + [lower_bound_amp],
    'lr': 1e-2,
    'weight_decay': 0},], betas=(0,0.9))

samples = get_samples(n_samples)
plt.figure(figsize=(5,5))
plt.plot(var_to_np(samples)[:,0],var_to_np(samples)[:,1], ls='', marker='o')
plt.plot(var_to_np(inputs)[:,0],var_to_np(inputs)[:,1], ls='', marker='o')
sliced_loss = sliced_from_samples(inputs, samples, 10, None)
print(sliced_loss)

from ampphase import outs_to_amp_phase, to_amp_phase, amp_phase_to_x_y

def set_phase_interval_around_mean(phases, mean_phase):
    mean_phase = mean_phase.unsqueeze(0)
    return ((phases - mean_phase + np.pi) % (2 * np.pi)) - np.pi + mean_phase


i_amp, i_phase = to_amp_phase(inputs[:,:1],inputs[:,1:])
i_amp = i_amp.clone().detach()
i_phase = i_phase.clone().detach()

In [ ]:
from reversible.gaussian import get_gauss_samples
from ampphase import amp_phase_to_x_y
from reversible.sliced import sliced_from_samples
n_samples = 500
n_epochs = 10000
for i_epoch in range(n_epochs):
    samples = get_samples(n_samples)
    mean_sample = (th.mean(samples, dim=0, keepdim=True) + th.mean(inputs, dim=0, keepdim=True)) / 2
    _, mean_phase = to_amp_phase(mean_sample[:,:1], mean_sample[:,1:])

    s_amp, s_phase = to_amp_phase(samples[:,:1], samples[:,1:])
    s_phase = set_phase_interval_around_mean(s_phase, mean_phase.squeeze())
    i_amp, i_phase = to_amp_phase(inputs[:,:1],inputs[:,1:])

    i_phase = set_phase_interval_around_mean(i_phase, mean_phase.squeeze())

    sliced_amp = sliced_from_samples(s_amp, i_amp, 10, None)
    sliced_phase = sliced_from_samples(s_phase, i_phase, 10, None)

    
    sliced_loss = sliced_amp + sliced_phase
    optim_dist.zero_grad()
    sliced_loss.backward()
    optim_dist.step()
    if i_epoch % (n_epochs // 20) == 0:
        print("sliced_loss: {:.2f}".format(sliced_loss.item()))
        print("sliced_amp: {:.2f}".format(sliced_amp.item()))
        print("sliced_phase: {:.2f}".format(sliced_phase.item()))
        print(means_per_cluster)
        print(th.exp(stds_per_cluster[0]))
        print(lower_bound_amp)
        fig = plt.figure(figsize=(3,3))
        plt.plot(var_to_np(samples)[:,0],var_to_np(samples)[:,1], ls='', marker='o')
        plt.plot(var_to_np(inputs)[:,0],var_to_np(inputs)[:,1], ls='', marker='o')
        display(fig)
        plt.close(fig)



### with DTF

In [ ]:
from reversible.uniform import get_uniform_samples
from disttransform import DistTransformResNet
def get_samples(n_samples):
    mean = means_per_cluster[0]
    std = th.exp(stds_per_cluster[0])

    amps = th.abs(get_gauss_samples(n_samples, mean[:1], std[:1], truncate_to=3))
    phases = get_uniform_samples(n_samples, mean[1:], std[1:] * 2 * np.pi)
    amp_phase = dtf(th.cat((amps, phases), dim=1))
    amps, phases = amp_phase[:,:1], amp_phase[:,1:]
    x, y = amp_phase_to_x_y(amps, phases)
    return th.cat((x,y), dim=1)

n_dims=2
n_clusters = 1
means_per_cluster = [th.autograd.Variable(th.zeros(n_dims), requires_grad=True)
                     for _ in range(n_clusters)]
# keep in mind this is in log domain so 0 is std 1
stds_per_cluster = [th.autograd.Variable(th.zeros(n_dims), requires_grad=True)
                    for _ in range(n_clusters)]

dtf = DistTransformResNet(2,2,200)
optim_dist = th.optim.Adam(
                          [
    {'params': means_per_cluster + stds_per_cluster + list(dtf.parameters()),
    'lr': 1e-2,
    'weight_decay': 0},], betas=(0,0.9))

samples = get_samples(n_samples)
plt.figure(figsize=(5,5))
plt.plot(var_to_np(samples)[:,0],var_to_np(samples)[:,1], ls='', marker='o')
plt.plot(var_to_np(inputs)[:,0],var_to_np(inputs)[:,1], ls='', marker='o')
sliced_loss = sliced_from_samples(inputs, samples, 10, None)
print(sliced_loss)

from ampphase import outs_to_amp_phase, to_amp_phase, amp_phase_to_x_y

def set_phase_interval_around_mean(phases, mean_phase):
    mean_phase = mean_phase.unsqueeze(0)
    return ((phases - mean_phase + np.pi) % (2 * np.pi)) - np.pi + mean_phase


i_amp, i_phase = to_amp_phase(inputs[:,:1],inputs[:,1:])
i_amp = i_amp.clone().detach()
i_phase = i_phase.clone().detach()

In [ ]:
from reversible.gaussian import get_gauss_samples
from ampphase import amp_phase_to_x_y
from reversible.sliced import sliced_from_samples
n_samples = 500
n_epochs = 10000
for i_epoch in range(n_epochs):
    samples = get_samples(n_samples)
    mean_sample = (th.mean(samples, dim=0, keepdim=True) + th.mean(inputs, dim=0, keepdim=True)) / 2
    _, mean_phase = to_amp_phase(mean_sample[:,:1], mean_sample[:,1:])

    s_amp, s_phase = to_amp_phase(samples[:,:1], samples[:,1:])
    s_phase = set_phase_interval_around_mean(s_phase, mean_phase.squeeze())
    i_amp, i_phase = to_amp_phase(inputs[:,:1],inputs[:,1:])

    i_phase = set_phase_interval_around_mean(i_phase, mean_phase.squeeze())

    sliced_amp = sliced_from_samples(s_amp, i_amp, 10, None)
    sliced_phase = sliced_from_samples(s_phase, i_phase, 10, None)

    
    sliced_loss = sliced_amp + sliced_phase
    optim_dist.zero_grad()
    sliced_loss.backward()
    optim_dist.step()
    if i_epoch % (n_epochs // 20) == 0:
        print("sliced_loss: {:.2f}".format(sliced_loss.item()))
        print("sliced_amp: {:.2f}".format(sliced_amp.item()))
        print("sliced_phase: {:.2f}".format(sliced_phase.item()))
        print(means_per_cluster)
        print(th.exp(stds_per_cluster[0]))
        fig = plt.figure(figsize=(3,3))
        plt.plot(var_to_np(samples)[:,0],var_to_np(samples)[:,1], ls='', marker='o')
        plt.plot(var_to_np(inputs)[:,0],var_to_np(inputs)[:,1], ls='', marker='o')
        display(fig)
        plt.close(fig)



In [ ]:
mean = means_per_cluster[0]
std = th.exp(stds_per_cluster[0])

amps = th.abs(get_gauss_samples(n_samples, mean[:1], std[:1], truncate_to=3))
phases = get_uniform_samples(n_samples, mean[1:], std[1:] * 2 * np.pi)
amp_phase = dtf(th.cat((amps, phases), dim=1))

plt.figure(figsize=(4,4))
plt.plot(np.sort(var_to_np(amps).squeeze()), np.sort(var_to_np(amp_phase[:,0]).squeeze()))
plt.figure(figsize=(4,4))
plt.plot(np.sort(var_to_np(phases).squeeze()), np.sort(var_to_np(amp_phase[:,1]).squeeze()))

In [ ]:
[th.max(th.abs(p)) for p in list(dtf.parameters())]

## without abs

In [ ]:
from reversible.uniform import get_uniform_samples
from disttransform import DistTransformResNet
def get_samples(n_samples):
    mean = means_per_cluster[0]
    std = th.exp(stds_per_cluster[0])

    amps = get_gauss_samples(n_samples, mean[:1], std[:1], truncate_to=3)
    phases = get_uniform_samples(n_samples, mean[1:], std[1:] * 2 * np.pi)
    amp_phase = dtf(th.cat((amps, phases), dim=1))
    amps, phases = amp_phase[:,:1], amp_phase[:,1:]
    x, y = amp_phase_to_x_y(amps, phases)
    return th.cat((x,y), dim=1)

n_dims=2
n_clusters = 1
means_per_cluster = [th.autograd.Variable(th.zeros(n_dims), requires_grad=True)
                     for _ in range(n_clusters)]
# keep in mind this is in log domain so 0 is std 1
stds_per_cluster = [th.autograd.Variable(th.zeros(n_dims), requires_grad=True)
                    for _ in range(n_clusters)]

dtf = DistTransformResNet(2,2,200)
optim_dist = th.optim.Adam(
                          [
    {'params': means_per_cluster + stds_per_cluster + list(dtf.parameters()),
    'lr': 1e-2,
    'weight_decay': 0},], betas=(0,0.9))

samples = get_samples(n_samples)
plt.figure(figsize=(5,5))
plt.plot(var_to_np(samples)[:,0],var_to_np(samples)[:,1], ls='', marker='o')
plt.plot(var_to_np(inputs)[:,0],var_to_np(inputs)[:,1], ls='', marker='o')
sliced_loss = sliced_from_samples(inputs, samples, 10, None)
print(sliced_loss)

from ampphase import outs_to_amp_phase, to_amp_phase, amp_phase_to_x_y

def set_phase_interval_around_mean(phases, mean_phase):
    mean_phase = mean_phase.unsqueeze(0)
    return ((phases - mean_phase + np.pi) % (2 * np.pi)) - np.pi + mean_phase


i_amp, i_phase = to_amp_phase(inputs[:,:1],inputs[:,1:])
i_amp = i_amp.clone().detach()
i_phase = i_phase.clone().detach()

In [ ]:
from reversible.gaussian import get_gauss_samples
from ampphase import amp_phase_to_x_y
from reversible.sliced import sliced_from_samples
n_samples = 500
n_epochs = 10000
for i_epoch in range(n_epochs):
    samples = get_samples(n_samples)
    mean_sample = (th.mean(samples, dim=0, keepdim=True) + th.mean(inputs, dim=0, keepdim=True)) / 2
    _, mean_phase = to_amp_phase(mean_sample[:,:1], mean_sample[:,1:])

    s_amp, s_phase = to_amp_phase(samples[:,:1], samples[:,1:])
    s_phase = set_phase_interval_around_mean(s_phase, mean_phase.squeeze())
    i_amp, i_phase = to_amp_phase(inputs[:,:1],inputs[:,1:])

    i_phase = set_phase_interval_around_mean(i_phase, mean_phase.squeeze())

    sliced_amp = sliced_from_samples(s_amp, i_amp, 10, None)
    sliced_phase = sliced_from_samples(s_phase, i_phase, 10, None)

    
    sliced_loss = sliced_amp + sliced_phase
    optim_dist.zero_grad()
    sliced_loss.backward()
    optim_dist.step()
    if i_epoch % (n_epochs // 20) == 0:
        print("sliced_loss: {:.2f}".format(sliced_loss.item()))
        print("sliced_amp: {:.2f}".format(sliced_amp.item()))
        print("sliced_phase: {:.2f}".format(sliced_phase.item()))
        print(means_per_cluster)
        print(th.exp(stds_per_cluster[0]))
        fig = plt.figure(figsize=(3,3))
        plt.plot(var_to_np(samples)[:,0],var_to_np(samples)[:,1], ls='', marker='o')
        plt.plot(var_to_np(inputs)[:,0],var_to_np(inputs)[:,1], ls='', marker='o')
        display(fig)
        plt.close(fig)



In [ ]:
mean = means_per_cluster[0]
std = th.exp(stds_per_cluster[0])

amps = th.abs(get_gauss_samples(n_samples, mean[:1], std[:1], truncate_to=3))
phases = get_uniform_samples(n_samples, mean[1:], std[1:] * 2 * np.pi)
amp_phase = dtf(th.cat((amps, phases), dim=1))

plt.figure(figsize=(4,4))
plt.plot(np.sort(var_to_np(amps).squeeze()), np.sort(var_to_np(amp_phase[:,0]).squeeze()))
plt.figure(figsize=(4,4))
plt.plot(np.sort(var_to_np(phases).squeeze()), np.sort(var_to_np(amp_phase[:,1]).squeeze()))

In [ ]:
from reversible.uniform import get_uniform_samples
from disttransform import DistTransformResNet
def get_samples(n_samples):
    mean = means_per_cluster[0]
    std = th.exp(stds_per_cluster[0])

    amps = get_gauss_samples(n_samples, mean[:1], std[:1], truncate_to=3)
    phases = get_uniform_samples(n_samples, mean[1:], std[1:] * 2 * np.pi)
    amp_phase = dtf(th.cat((amps, phases), dim=1))
    amps, phases = amp_phase[:,:1], amp_phase[:,1:]
    x, y = amp_phase_to_x_y(amps, phases)
    return th.cat((x,y), dim=1)

n_dims=2
n_clusters = 1
means_per_cluster = [th.autograd.Variable(th.zeros(n_dims), requires_grad=True)
                     for _ in range(n_clusters)]
# keep in mind this is in log domain so 0 is std 1
stds_per_cluster = [th.autograd.Variable(th.zeros(n_dims), requires_grad=True)
                    for _ in range(n_clusters)]

dtf = DistTransformResNet(2,2,200)
optim_dist = th.optim.Adam(
                          [
    {'params': means_per_cluster + stds_per_cluster + list(dtf.parameters()),
    'lr': 1e-2,
    'weight_decay': 0},], betas=(0,0.9))

samples = get_samples(n_samples)
plt.figure(figsize=(5,5))
plt.plot(var_to_np(samples)[:,0],var_to_np(samples)[:,1], ls='', marker='o')
plt.plot(var_to_np(inputs)[:,0],var_to_np(inputs)[:,1], ls='', marker='o')
sliced_loss = sliced_from_samples(inputs, samples, 10, None)
print(sliced_loss)

from ampphase import outs_to_amp_phase, to_amp_phase, amp_phase_to_x_y

def set_phase_interval_around_mean(phases, mean_phase):
    mean_phase = mean_phase.unsqueeze(0)
    return ((phases - mean_phase + np.pi) % (2 * np.pi)) - np.pi + mean_phase


i_amp, i_phase = to_amp_phase(inputs[:,:1],inputs[:,1:])
i_amp = i_amp.clone().detach()
i_phase = i_phase.clone().detach()

In [ ]:
from reversible.gaussian import get_gauss_samples
from ampphase import amp_phase_to_x_y
from reversible.sliced import sliced_from_samples
n_samples = 500
n_epochs = 1000
for i_epoch in range(n_epochs):
    samples = get_samples(n_samples)
    sliced_loss = sliced_from_samples(inputs, samples, 10, None)
    optim_dist.zero_grad()
    sliced_loss.backward()
    optim_dist.step()
    if i_epoch % (n_epochs // 20) == 0:
        print("sliced_loss: {:.2f}".format(sliced_loss.item()))
        print(means_per_cluster)
        print(th.exp(stds_per_cluster[0]))
        fig = plt.figure(figsize=(3,3))
        plt.plot(var_to_np(samples)[:,0],var_to_np(samples)[:,1], ls='', marker='o')
        plt.plot(var_to_np(inputs)[:,0],var_to_np(inputs)[:,1], ls='', marker='o')
        display(fig)
        plt.close(fig)

In [ ]:
# separable
# 

In [ ]:
mean = means_per_cluster[0]
std = th.exp(stds_per_cluster[0])

amps = th.abs(get_gauss_samples(n_samples, mean[:1], std[:1], truncate_to=3))
phases = get_uniform_samples(n_samples, mean[1:], std[1:] * 2 * np.pi)
amp_phase = dtf(th.cat((amps, phases), dim=1))

plt.figure(figsize=(4,4))
plt.plot(np.sort(var_to_np(amps).squeeze()), np.sort(var_to_np(amp_phase[:,0]).squeeze()))
plt.figure(figsize=(4,4))
plt.plot(np.sort(var_to_np(phases).squeeze()), np.sort(var_to_np(amp_phase[:,1]).squeeze()))

In [ ]:
#reparametrize folded, parametrize mean as 